In [29]:
import pandas as pd
import numpy as np
import os
from pathlib import Path       
from datetime import datetime 


In [45]:
base_dir = os.getcwd()
base_dir

'/home/sobottka/BSE/Master_Thesis/bse-thesis-synthetic-data/data/processed_files'

In [46]:
os.listdir()

['SHANGHAI_processed.csv',
 'BOVESPA_processed.csv',
 'MSCI_processed.csv',
 'FTSE_JSE_processed.csv',
 'NIFTY50_processed.csv']

In [4]:
next_folder = os.chdir('data/processed_files')

In [5]:
current_path = os.getcwd()
current_path

'/home/sobottka/BSE/Master_Thesis/bse-thesis-synthetic-data/data/processed_files'

In [47]:
files = os.listdir()
train_dfs = {}

for i in files: 
    key_name = f"train_pd_{i.split('_')[0]}"
    train_dfs[key_name] = pd.read_csv(i)

train_dfs

{'train_pd_SHANGHAI':             Date    Price      Open      High       Low    Vol. Change %  \
 0     01/02/2024  2962.28  2,972.77  2,976.27  2,962.28  30.41B   -0.43%   
 1     01/02/2025  3262.56  3,347.94  3,351.72  3,242.09  56.14B   -2.66%   
 2     01/03/2023  3116.51  3,087.51  3,119.86  3,073.05  28.14B    0.88%   
 3     01/03/2024  2967.25  2,957.11  2,971.28  2,953.29  28.55B    0.17%   
 4     01/03/2025  3211.43  3,267.08  3,273.57  3,205.78  51.76B   -1.57%   
 ...          ...      ...       ...       ...       ...     ...      ...   
 1207  12/30/2022  3089.26  3,084.52  3,096.31  3,082.20  21.75B    0.51%   
 1208  12/30/2024  3407.33  3,395.40  3,412.84  3,394.96  45.53B    0.21%   
 1209  12/31/2020  3473.07  3,419.73  3,474.92  3,419.73  33.57B    1.72%   
 1210  12/31/2021  3639.78  3,626.24  3,642.84  3,624.94  32.97B    0.57%   
 1211  12/31/2024  3351.76  3,406.97  3,413.45  3,351.76  50.27B   -1.63%   
 
       LogReturn  
 0           NaN  
 1      0.09655

In [48]:
train_dfs.keys()

dict_keys(['train_pd_SHANGHAI', 'train_pd_BOVESPA', 'train_pd_MSCI', 'train_pd_FTSE', 'train_pd_NIFTY50'])

In [49]:
train_dfs['train_pd_SHANGHAI'].head(1000)

,Date,Price,Open,High,Low,Vol.,Change %,LogReturn
0,01/02/2024,2962.28,"2,972.77","2,976.27","2,962.28",30.41B,-0.43%,NaN
1,01/02/2025,3262.56,"3,347.94","3,351.72","3,242.09",56.14B,-2.66%,0.096553
2,01/03/2023,3116.51,"3,087.51","3,119.86","3,073.05",28.14B,0.88%,-0.045798
3,01/03/2024,2967.25,"2,957.11","2,971.28","2,953.29",28.55B,0.17%,-0.049078
4,01/03/2025,3211.43,"3,267.08","3,273.57","3,205.78",51.76B,-1.57%,0.079081
...,...,...,...,...,...,...,...,...
995,11/01/2023,3023.08,"3,038.18","3,038.33","3,013.93",28.01B,0.14%,0.017984
996,11/01/2024,3272.01,"3,275.58","3,305.32","3,258.73",85.21B,-0.24%,0.079128
997,11/02/2020,3225.12,"3,228.72","3,242.80","3,209.91",22.68B,0.02%,-0.014434
998,11/02/2021,3505.63,"3,543.38","3,559.05","3,477.66",41.76B,-1.10%,0.083400



### Time Handling

Sort by stock_id, time_id (or timestamp). This is critical for rolling features later.

Normalize time:

    Extract useful time features: second_in_day, minute_of_day.

    You’ll need these for intraday seasonality (prices behave differently at open/close).


In [50]:
for key, df in train_dfs.items():
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.set_index("Date").sort_index()
    train_dfs[key] = df  # reassign the modified dataframe


In [51]:
train_dfs['train_pd_BOVESPA']

,Price,Open,High,Low,Vol.,Change %,LogReturn
Date,,,,,,,
2020-10-01,95479,"94,604","95,486","93,599",9.24M,0.93%,-0.426322
2020-10-02,94016,"95,475","95,996","93,897",8.58M,-1.53%,-0.436828
2020-10-05,96089,"94,019","96,414","93,984",8.62M,2.20%,-0.315950
2020-10-06,95615,"96,091","97,405","95,211",11.30M,-0.49%,-0.169568
2020-10-07,95526,"95,616","96,380","94,881",9.83M,-0.09%,-0.178290
...,...,...,...,...,...,...,...
2025-09-25,145306,"146,492","146,519","145,187",8.29B,-0.81%,0.099181
2025-09-26,145447,"145,326","146,235","145,146",6.34B,0.10%,0.089387
2025-09-29,146337,"145,447","147,558","145,447",7.07B,0.61%,0.227463


In [52]:
train_dfs['train_pd_BOVESPA'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1246 entries, 2020-10-01 to 2025-10-01
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      1246 non-null   int64  
 1   Open       1246 non-null   object 
 2   High       1246 non-null   object 
 3   Low        1246 non-null   object 
 4   Vol.       1246 non-null   object 
 5   Change %   1246 non-null   object 
 6   LogReturn  1245 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 77.9+ KB


In [53]:
#Checking for duplicates

print(train_dfs['train_pd_BOVESPA'].duplicated())

Date
2020-10-01    False
2020-10-02    False
2020-10-05    False
2020-10-06    False
2020-10-07    False
              ...  
2025-09-25    False
2025-09-26    False
2025-09-29    False
2025-09-30    False
2025-10-01    False
Length: 1246, dtype: bool


### Missing Values (Context-aware)

Prices (*_price): forward-fill within each (stock_id, date_id); if the very first row is missing, back-fill from the first available in that day, else drop that row (rare).

Sizes (*_size): forward-fill; if a day starts with NaNs, set to 0 only if your EDA showed that means “no displayed size” rather than missing.

Flags (imbalance_buy_sell_flag): treat 0 as “no imbalance,” not missing. If NaN exists, map NaN→0 after checking frequency.

Keep a mask column was_nan_any for later error analysis (missingness can be predictive in microstructure).




In [54]:
# Different methods to count the number of the rows
for key, df in train_dfs.items():

    print(key, len(df)) #number of rows
    print(key, df.shape[0]) # number of rows, shape [1] give the number of columns
    print(key, df.count() ) # counts non-NAN values per column
    print(key, df.count().sum() )#counts all non-NAN values in the whole DataFrame

train_pd_SHANGHAI 1212
train_pd_SHANGHAI 1212
train_pd_SHANGHAI Price        1212
Open         1212
High         1212
Low          1212
Vol.         1212
Change %     1212
LogReturn    1211
dtype: int64
train_pd_SHANGHAI 8483
train_pd_BOVESPA 1246
train_pd_BOVESPA 1246
train_pd_BOVESPA Price        1246
Open         1246
High         1246
Low          1246
Vol.         1246
Change %     1246
LogReturn    1245
dtype: int64
train_pd_BOVESPA 8721
train_pd_MSCI 1256
train_pd_MSCI 1256
train_pd_MSCI Price        1256
Open         1256
High         1256
Low          1256
Vol.         1256
Change %     1256
LogReturn    1255
dtype: int64
train_pd_MSCI 8791
train_pd_FTSE 1250
train_pd_FTSE 1250
train_pd_FTSE Price        1250
Open         1250
High         1250
Low          1250
Vol.         1231
Change %     1250
LogReturn    1249
dtype: int64
train_pd_FTSE 8730
train_pd_NIFTY50 1242
train_pd_NIFTY50 1242
train_pd_NIFTY50 Price        1242
Open         1242
High         1242
Low          1242

### Handling Outliers

**Identify Which Columns Need Outlier Handling**

For Optiver TAC dataset, focus on numeric columns that affect price movement predictions:

Prices: bid_price, ask_price, reference_price, wap, far_price, near_price

Sizes / volumes: bid_size, ask_size, matched_size, imbalance_size

Derived features (later in feature engineering): spreads, ratios, etc.


**Decide How to Detect Outliers**

**Statistical Methods**

    1. Percentile clipping (Winsorizing)
        lower = df[col].quantile(0.005)
        upper = df[col].quantile(0.995)
        df[col] = df[col].clip(lower, upper)


    2. Z-score Method
        Calculate z = (x - mean) / std within a stock or day.

        Cap |z| > 3 or 4.

        Useful if distributions differ across stocks.

    3. IQR (Interquantile Range)
        Q1 = df[col].quantile(0.25)

        Q3 = df[col].quantile(0.75)

        Outliers: x < Q1 - 1.5*IQR or x > Q3 + 1.5*IQR

        Sometimes better for skewed distributions.


**Contextual / Market-Specific Methods**

    Bid / Ask Price Constraints

        Ask ≥ Bid always.

        Reference price usually close to WAP; if WAP deviates >5-10% from reference, treat as outlier.

    Imbalance / Matched Size

        If imbalance_size is negative when flag = 1, it’s likely an error.

        Cap extreme volumes to avoid skewing features like imbalance ratios.

    Time-Series Smoothness

        Sudden jump in wap or bid_price > X% in one second might be a misrecord; smooth or clip.

**Implementation Steps**
    1. Group by stock_id and optionally date_id

        Outliers are relative to each stock; one stock’s price might be 1000x another’s.
    
    2. Compute per-stock statistics

        group = df.groupby('stock_id')
        df['wap_clipped'] = group['wap'].transform(lambda x: x.clip(x.quantile(0.005), x.quantile(0.995)))

    3. Decide on treatment

        Clip / Winsorize → preserves row, reduces influence.

        Set to NaN / interpolate → can be done if value is obviously wrong.

        Drop row → rarely, if the row is corrupted across many columns.

**Practical Tips for Quant Modeling**

    Always clip based on training data stats, not the test set, to avoid leakage.

    Create indicator columns for extreme values → sometimes “being an outlier” is predictive.

    Visualize distributions after clipping:

        import seaborn as sns
        sns.boxplot(x='stock_id', y='wap', data=df.sample(5000))


| Step                                                             | Method                         | Notes                                         |
| ---------------------------------------------------------------- | ------------------------------ | --------------------------------------------- |
| Prices (`wap`, `bid_price`, `ask_price`)                         | Percentile clipping 0.5%-99.5% | Preserve order book structure                 |
| Sizes (`bid_size`, `ask_size`, `matched_size`, `imbalance_size`) | Winsorize or cap               | Avoid extreme ratios dominating               |
| Far / Near prices                                                | Missing + clip                 | Only if used in features                      |
| Indicator for clipped values                                     | Binary flag                    | Could help ML model recognize unusual events  |
| Temporal smoothing                                               | Optional                       | If single-second jump is extreme, interpolate |


**Jane Street / Optiver mindset:**

   - You never just “remove outliers” blindly.

   - Ask: Is this extreme value a market reality (volatility) or an error in data?

   - Preserve real market extremes — they are informative signals. Only remove obvious errors.

In [26]:
price_columns = ['wap', 'bid_price', 'ask_price']

for col in price_columns: 
    lower = train_sorted_pd[col].quantile(0.005)
    upper = train_sorted_pd[col].quantile(0.995)
    train_sorted_pd[col] = train_sorted_pd[col].clip(lower, upper)

/tmp/ipykernel_1598690/3082504089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col] = train_sorted_pd[col].clip(lower, upper)
/tmp/ipykernel_1598690/3082504089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col] = train_sorted_pd[col].clip(lower, upper)
/tmp/ipykernel_1598690/3082504089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

**Winsorizing: statistical term, usually symmetric, replaces with boundary values.**

**Percentile Clipping: coding term, you specify quantile thresholds (not always symmetric).**

1. What is Winsorizing?

Winsorizing means replacing extreme values with the nearest acceptable percentile boundary, instead of dropping them.

Example (1%–99% winsorization):

If a value < 1st percentile → replace it with the 1st percentile.

If a value > 99th percentile → replace it with the 99th percentile.

So the distribution is “squished” at the tails, but no rows are removed.

🔹 2. What is Capping / Clipping?

Capping (a.k.a. clipping) is essentially the same as winsorizing in implementation:

You set a hard min and max (usually based on quantiles).

Any values outside are set equal to the boundaries.

The difference is mainly terminology:

In statistics, “winsorizing” usually refers to symmetric tail treatment (like 5%–95%).

In coding (pandas, numpy), you’ll usually see clip(), which is equivalent.

🔹 3. Why Use Winsorization/Capping for Size Columns?

Size features:

bid_size, ask_size, matched_size, imbalance_size
tend to have heavy-tailed distributions — most orders are small, but occasionally there are massive block trades.

If you don’t cap, a single huge bid_size could dominate your model (especially linear models).

But you don’t want to delete these rows, because the fact that a large order appeared might be informative.

Winsorizing preserves the row, just prevents extreme distortion.

In [27]:
size_cols = ['bid_size', 'ask_size', 'matched_size', 'imbalance_size']

for col in size_cols:
    lower = train_sorted_pd[col].quantile(0.01)   # 1st percentile
    upper = train_sorted_pd[col].quantile(0.99)   # 99th percentile
    
    train_sorted_pd[col + '_winsor'] = train_sorted_pd[col].clip(lower, upper)
    
    # Optional: indicator for whether it was capped
    train_sorted_pd[col + '_is_capped'] = (
        (train_sorted_pd[col] < lower) | (train_sorted_pd[col] > upper)
    ).astype(int)


/tmp/ipykernel_1598690/971069179.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col + '_winsor'] = train_sorted_pd[col].clip(lower, upper)
/tmp/ipykernel_1598690/971069179.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col + '_is_capped'] = (
/tmp/ipykernel_1598690/971069179.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 


### Memory Optimization

Datasets are huge in Optiver, so:

    Downcast columns (float64 → float32, int64 → int32).

    Use category dtype for stock_id.

    Save intermediate results as feather/parquet instead of CSV.


In [28]:
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.float32)
        else:
            df[col] = df[col].astype('category')
    return df





### Split Train/Valid/Test

Since this is time series:

    Split by time_id (or date), not randomly.

    Example: Train on first 80% of time blocks, validate on next 10%, test on last 10%.

    (Later, use rolling CV for more robustness).

Train → model learns.

Validation → tune hyperparameters.

Test → final unbiased evaluation.

How (time-series case):

Do not shuffle!

Split by time (earlier dates = train, middle = valid, latest = test).

1. Purpose of Each Split

Train set → Used to fit the model parameters (e.g., regression weights, neural net weights).

Validation set → Used to tune hyperparameters (things you don’t directly fit, like learning rate, number of layers, window size, regularization strength). You look at performance on validation data to decide which model configuration is “best.”

Test set → Used once at the very end for an unbiased performance estimate. It mimics future unseen data.

2. Why We Need Validation

If you only had train/test:

You’d try different hyperparameters, but each time you’d be peeking at the test set.

This makes the test set “contaminated” because you’re indirectly fitting to it.

So the validation set is a sandbox where you’re allowed to try things. The test is your final exam where you can’t look ahead.

3. Time Series Specifics

In time series, you cannot shuffle because future must never leak into the past.

Train → earlier dates (model learns patterns).

Validation → the “next chunk” of time. You try hyperparameters and see how the model forecasts.

Test → the most recent chunk. Used only at the end for unbiased evaluation.

In [ ]:
# Need to be checked

train = df[df['date'] < '2017-01-01']
valid = df[(df['date'] >= '2017-01-01') & (df['date'] < '2018-01-01')]
test  = df[df['date'] >= '2018-01-01']



### Pipeline Setup

Organize preprocessing so you can reproduce it later:

    Write functions: clean_missing(df), normalize(df), align_data(ob, tr).

    Save processed datasets for reuse.
